# Before demo

- Install `Python 3.10`
- Create virtual environment `python -m venv .venv`
- Activate virtual env `.venv\Scripts\activate` (Windows) or `source .venv/bin/activate` Linux/macOS
- Update base packages: `python -m pip install -U pip setuptools wheel`
- Add required packages: `python -m pip install -r requirements.txt`
- (Optional) Start data proxy: `python .\hackathon_1\webserver.py`
- run `jupyter lab`

# Demo

**Get data from Infrabel's opendata portal**
- Go to [https://opendata.infrabel.be](https://opendata.infrabel.be) (+ introduce the portal)
- Click on "*Infrastructure*"
- Search for "*Energy*" (left bar)
- Click on the dataset "*Monthly traction energy consumption (distributed)*"
- Introduce the data, click on "Table" to show the data
- Click on "*Export*"
- We we use the **CSV** data format (**copy the csv url**)

**Create a new jupyter notebook (_analysis.ipynb_)**

- Import the libraries

In [ ]:
import infrabel_demo

import pandas as pd  # Fetch and manipulate data
import geopandas as gpd # Add geographic capabilities to pandas
import pygwalker as pyg  # Quick data profiling  analysis
import duckdb  # Manipulate data in SQL
import folium # Plot maps
import branca # Colors for maps

# Prevent warnings when inplace copying
pd.options.mode.copy_on_write = True

## 1. Getting energy data

Show how to pull data with pandas, and display it

In [ ]:
energy = pd.read_csv("https://opendata.infrabel.be/api/explore/v2.1/catalog/datasets/maandelijks-tractie-energieverbruik-met-verdeling/exports/csv?lang=en&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B", sep=';')

In [ ]:
energy

## 2. Profiling data

**Profiling data**: Determine the quality of the data (null, gaps, etc), and get an overall look / doing a quick analysis of what it contains.

**Missing values**:
- Go to "*Data*" Tab
- See missing values (*null*)
- Notice the missing values are for dates in the past
- Notice **we'll most likely exclude the nulls, but we need to be sure they are no null elsewhere**

**Quick analysis - Energy consumption over time (Passengers)**
- Go to "*Visualization tab*"
- Goal: *Taking a look at energy consumption data over time*
- Change "*Month*"'s "*Semantic Type*" to "*Temporal*"
- Drag and drop "*Month*" to "*X-Axis*"
- Drag and drop "*Passengers*" to "*Y-Axis*"
- Change "Layout Mode" to "Container"
- **Analysis**: Constent passengers trains over time. No data before 2009.

**Refined analysis - Energy consumption over time (Freight)**
- Drag and drop "*Freight*" to "*X-Axis*"
- **Analysis**: Freight energy consumption is decreasing over the year. Maybe because we have more efficient trains ? Maybe because we carry less freight ? -> To be investigated (with other Infrabel data)

**Refined analysis - Energy consumption over time (High Speed Trains)**
- Drag and drop "*High seed train*" to "*X-Axis*"
- **Analysis**: Gap in 2020-2021 (Corona / lockdown), energy consuption is increasing over years. Maybe because we have less efficient trains ? Maybe because we have more TGV passing by Belgium ? -> To be investigated (with other Infrabel data)

In [ ]:
walker = pyg.walk(energy)

# 3. Cleaning data

Show how to clean data (remove *nulls*) in Python (pandas) and in SQL (duckdb).

- For those who don't know pandas, better learning duckdb directly
  - **SQL is a valuable skill, usable in many situation. Pandas syntax is useless outside Python.**

In [ ]:
energy[energy['Passengers'].notnull()]

In [ ]:
duckdb.query(
    """SELECT * FROM energy
    WHERE Passengers IS NOT NULL"""
).to_df()

# 3. Plotting maps

In [ ]:
lines = gpd.read_file("https://opendata.infrabel.be/api/explore/v2.1/catalog/datasets/geosporen/exports/geojson?lang=en&timezone=Europe%2FBerlin")

In [ ]:
lines

In [ ]:
lines.plot()

# Plotting an intervactive map

1. Plotting folium map
2. Transform data
3. Replot

 - `folium.Map()`
 - `folium.Map(location=[50.425, 4.34])`
 - `folium.Map(location=[50.425, 4.34], zoom_start=8)`
 - `folium.Map(location=[50.425, 4.34], zoom_start=8, tiles="CartoDB Positron")`

In [ ]:
folium.Map(location=[50.425, 4.34], tiles="CartoDB Positron", zoom_start=8)

Remove empty geometries

In [ ]:
lines = lines.dropna(subset=["geometry"])

In [ ]:
lines['trackcode'].unique()

Calculate length of each line section

In [ ]:
# Calculate the length using the Lambert 72 Projection (EPSG 31370)
lines['length_km'] = lines['geometry'].to_crs('EPSG:31370').length / 1000

In [ ]:
# ax = lines.plot(figsize=(15, 10), alpha=0.5, edgecolor='k')
# cx.add_basemap(ax, crs=lines.crs, source=cx.providers.CartoDB.Positron)
# plt.show()

    # folium.Popup(row["id"]).add_to(geojson)

In [ ]:
lines['modifdate'] = lines['modifdate'].astype(str)

In [ ]:
line_map = folium.Map(location=[50.425, 4.34], tiles="CartoDB Positron", zoom_start=8)

colormap = branca.colormap.LinearColormap(
    colors=['green', 'yellow', 'red'],
    vmin=lines['length_km'].min(),
    vmax=lines['length_km'].max()
)

folium.GeoJson(
    lines.to_geo_dict(),
    style_function=lambda x: {"color": colormap(x['properties']['length_km'])}
).add_to(line_map)

line_map